In [1]:
# 패키지 임포트
import numpy as np
import pandas as pd
import scipy.stats as ss
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
# !pip install plotly
import plotly.offline as off
import plotly.graph_objs as go
pd.set_option('display.max_columns', 500)

plt.rc("font", family="Malgun Gothic") # matplotlib 한글깨짐 방지

In [5]:
# 데이터 로드
# 부동산 원본 데이터
estate_price_2018 = pd.read_csv('estate_price_2018.csv', encoding='cp949')
estate_price_2019 = pd.read_csv('estate_price_2019.csv', encoding='cp949')
estate_price_2020 = pd.read_csv('estate_price_2020.csv', encoding='cp949')

# train data 생성 위한 전처리 수행
train_data = pd.concat([estate_price_2018, estate_price_2019, estate_price_2020])
train_data.drop(['해제사유발생일', '거래유형', '중개사소재지'], inplace=True, axis=1)
train_data.drop(['번지', '본번', '부번','도로명'], inplace=True, axis=1)
train_data.rename(columns = {'전용면적(㎡)':'전용면적'}, inplace=True)
# 결측치 없음
# 중복 삭제
train_data.drop_duplicates(inplace=True)

# 이상치 삭제 
train_data.drop(train_data.loc[train_data['층']<0].index, inplace=True)
train_data.drop(train_data.loc[train_data['전용면적']>300].index, inplace=True)

# train_data_original 생성
train_data.to_csv('train_data_original.csv', encoding='utf-8-sig', index=False)

In [9]:
train_original_data=pd.read_csv('train_data_original.csv')
train_original_data

,시군구,단지명,전용면적,계약년월,계약일,거래금액(만원),층,건축년도
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,3,"130,000",4,1987
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,8,"117,000",2,1987
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,11,"130,000",1,1987
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,19,"139,500",2,1987
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,5,"107,500",5,1987
...,...,...,...,...,...,...,...,...
237982,서울특별시 중랑구 중화동,한영(104),106.21,202003,11,"45,400",6,2003
237983,서울특별시 중랑구 중화동,한영(104),67.57,202008,7,"26,000",2,2003
237984,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,10,"44,000",6,2007
237985,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,3,"54,800",5,2007


In [10]:
train_data=pd.read_csv('train_data_ver1.csv')
train_data

,전용면적,거래금액,층,건축년도,제곱미터단가,구,동,단지명브랜드,계약날짜,계약날짜인코딩,동별 공원 갯수,8학군,동별지하철역수,구별 교과학원 갯수
0,79.97,130000,4,1987,1625.609604,강남구,개포동,기타,2018-01-03,2,16.0,1,1,20952
1,79.97,117000,2,1987,1463.048643,강남구,개포동,기타,2018-01-08,7,16.0,1,1,20952
2,79.97,130000,1,1987,1625.609604,강남구,개포동,기타,2018-01-11,10,16.0,1,1,20952
3,79.97,139500,2,1987,1744.404152,강남구,개포동,기타,2018-03-19,77,16.0,1,1,20952
4,54.98,107500,5,1987,1955.256457,강남구,개포동,기타,2018-04-05,94,16.0,1,1,20952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237982,106.21,45400,6,2003,427.455042,중랑구,중화동,기타,2020-03-11,800,0.0,0,1,1997
237983,67.57,26000,2,2003,384.786148,중랑구,중화동,기타,2020-08-07,949,0.0,0,1,1997
237984,95.94,44000,6,2007,458.619971,중랑구,중화동,현대,2020-07-10,921,0.0,0,1,1997
237985,100.17,54800,5,2007,547.069981,중랑구,중화동,현대,2020-12-03,1067,0.0,0,1,1997


In [34]:
# 단지명브랜드별 거래 횟수 대비 신축 거래 횟수
# train_original_data / train_data
danji_count=train_data['단지명브랜드'].value_counts()
sinchook=train_data.loc[train_data['건축년도']>2010] #신축
goochook=train_data.loc[train_data['건축년도']<=2010] #구축

sinchook_count=sinchook['단지명브랜드'].value_counts()
singoo=pd.concat([sinchook_count,danji_count], axis=1, keys=['sinchook_count','danji_count']).fillna(0)
singoo

,sinchook_count,danji_count
기타,16458.0,148730
래미안,5063.0,13872
힐스테이트,2967.0,5388
자이,2370.0,4166
푸르지오,1771.0,5734
이편한|e편한|e-편한,1387.0,4435
롯데캐슬,1135.0,3891
아이파크,1126.0,3086
두산|두산위브,649.0,5107
센트레빌,598.0,2133


In [46]:
# 단지 거래 횟수별 신축 비율
singoo['singoo_ratio']=singoo['sinchook_count']/singoo['danji_count']
singoo['singoo_ratio']
# singoo.loc[singoo['singoo_ratio']!=1]

기타              0.110657
래미안             0.364980
힐스테이트           0.550668
자이              0.568891
푸르지오            0.308859
이편한|e편한|e-편한    0.312740
롯데캐슬            0.291699
아이파크            0.364874
두산|두산위브         0.127080
센트레빌            0.280356
포스코|더샵          0.375986
데시앙             0.389714
sk|SK|에스케이      0.132709
현대              0.016495
하늘채             0.673469
리슈빌             1.000000
서희|서희스타힐스       0.977273
삼성              0.021814
스위첸             0.829060
호반              1.000000
한라|한라비발디        0.300000
GS              0.358209
KCC             0.317568
한화              0.037952
쌍용              0.014109
대우              0.014023
플래티넘            0.193548
우미              0.200000
벽산              0.000768
코오롱             0.000000
롯데              0.000000
벽산블루밍           0.000000
태영              0.000000
동부              0.000000
코아루             0.000000
계룡              0.000000
동문              0.000000
Name: singoo_ratio, dtype: float64

In [44]:
# 인코딩 데이터에 합치기
train_data['단지명별 신축거래비율']=train_data['단지명브랜드'].map(singoo['singoo_ratio'])
train_data[['단지명브랜드','단지명별 신축거래비율']].value_counts()

단지명브랜드        단지명별 신축거래비율
기타            0.110657       148730
현대            0.016495        17702
래미안           0.364980        13872
푸르지오          0.308859         5734
힐스테이트         0.550668         5388
두산|두산위브       0.127080         5107
삼성            0.021814         4630
이편한|e편한|e-편한  0.312740         4435
자이            0.568891         4166
벽산            0.000768         3905
롯데캐슬          0.291699         3891
아이파크          0.364874         3086
sk|SK|에스케이    0.132709         2351
대우            0.014023         2282
쌍용            0.014109         2268
센트레빌          0.280356         2133
포스코|더샵        0.375986         1141
한화            0.037952         1133
데시앙           0.389714          875
코오롱           0.000000          822
롯데            0.000000          728
벽산블루밍         0.000000          687
태영            0.000000          633
동부            0.000000          546
하늘채           0.673469          343
한라|한라비발디      0.300000          180
서희|서희스타힐스     0.977273          176
리슈

In [81]:
train_data.to_csv('train_data_ver2.csv', encoding='utf-8-sig')
print('train_data_ver2.csv가 저장되었습니다.')

train_data_ver2.csv가 저장되었습니다.
